### Adaptation from the Keras example
### Many parts of the code are carried over (re use maybe later)

In [4]:
!pip install tensorflow pandas matplotlib scikit-learn

  Using cached matplotlib-3.7.5-cp38-cp38-macosx_11_0_arm64.whl.metadata (5.7 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 2.0 MB/s eta 0:00:00a 0:00:01
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 4.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.4/232.4 kB 4.3 MB/s eta 0:00:00a 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 4.8 MB/s eta 0:00:0000:0100:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 3.1 MB/s eta 0:00:00
Using cached pyparsing-3.1.2-py3-non

In [3]:
import pandas as pd
import keras
from keras import layers
# from keras import ops
import tensorflow as tf
import numpy as np
import imageio
# from tensorflow_docs.vis import embed
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [4]:
# Configurations
batch_size = 64
num_channels = 3  # RGB images have 3 channels
num_classes = 2
image_size = (128, 96)  # Resize to save memory
latent_dim = 128

# Paths to data
image_dir = '../Data/HAM/ham_images'  # Directory containing images
csv_path = '../Experiments/two_class_metadata.csv'  # Path to CSV file

In [5]:
# Load data from CSV
df = pd.read_csv(csv_path)

# Extract image IDs and classes
image_ids = df['image_id'].values
labels = df['class'].values

# Encode labels to integers then to one-hot
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
one_hot_labels = to_categorical(integer_encoded, num_classes=num_classes)

In [6]:
# Define function to get full image path
def get_image_path(image_id):
    return f"{image_dir}/{image_id}.jpg" 

# Load and process images
def process_image(image_id):
    full_path = get_image_path(image_id)
    img = load_img(full_path, target_size=image_size, color_mode='rgb')  # Load as RGB
    img_array = img_to_array(img).astype("float32") / 255.0
    return img_array

In [7]:
# Create datasets
all_images = np.array([process_image(image_id) for image_id in image_ids])
all_labels = one_hot_labels

# Ensure the images have 4 dimensions (batch_size, height, width, channels)
# RGB images already have 3 channels, no need for additional dimension
all_images = np.reshape(all_images, (-1, image_size[0], image_size[1], num_channels))

# Create tf.data.Dataset
dataset = tf.data.Dataset.from_tensor_slices((all_images, all_labels))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

# Print data shapes for confirmation
print(f"Shape of images: {all_images.shape}")
print(f"Shape of labels: {all_labels.shape}")

generator_in_channels = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes

print(f"Generator input channels: {generator_in_channels}")
print(f"Discriminator input channels: {discriminator_in_channels}")

Shape of images: (10015, 128, 96, 3)
Shape of labels: (10015, 2)
Generator input channels: 130
Discriminator input channels: 5


In [8]:
# Create the discriminator.
discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((*image_size, discriminator_in_channels)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)

# Create the generator.
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        layers.Dense((image_size[0] // 4) * (image_size[1] // 4) * generator_in_channels),
        layers.LeakyReLU(negative_slope=0.2),
        layers.Reshape((image_size[0] // 4, image_size[1] // 4, generator_in_channels)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.Conv2D(num_channels, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

TypeError: ('Keyword argument not understood:', 'negative_slope')

In [ ]:
class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.seed = 1337
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data):
        # Unpack the data.
        real_images, one_hot_labels = data
        print(f"Real images shape: {real_images.shape}")
        print(f"One-hot labels shape: {one_hot_labels.shape}")

        # Ensure one_hot_labels is float32
        one_hot_labels = tf.cast(one_hot_labels, tf.float32)
        
        # Expand labels to match the shape of images for concatenation.
        image_one_hot_labels = tf.reshape(one_hot_labels, (-1, 1, 1, num_classes))
        image_one_hot_labels = tf.tile(image_one_hot_labels, [1, image_size[0], image_size[1], 1])
        print(f"Image one-hot labels shape: {image_one_hot_labels.shape}")

        # Sample random points in the latent space and concatenate the labels.
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim), seed=self.seed)
        random_vector_labels = tf.concat([random_latent_vectors, one_hot_labels], axis=1)
        print(f"Random latent vectors shape: {random_latent_vectors.shape}")
        print(f"Random vector labels shape: {random_vector_labels.shape}")

        # Decode the noise (guided by labels) to fake images.
        generated_images = self.generator(random_vector_labels)
        print(f"Generated images shape: {generated_images.shape}")

        # Ensure image_one_hot_labels is float32
        image_one_hot_labels = tf.cast(image_one_hot_labels, tf.float32)

        # Combine them with real images. Note that we are concatenating the labels
        # with these images here.
        fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], axis=-1)
        real_image_and_labels = tf.concat([real_images, image_one_hot_labels], axis=-1)
        combined_images = tf.concat([fake_image_and_labels, real_image_and_labels], axis=0)
        print(f"Combined images shape: {combined_images.shape}")

        # Assemble labels discriminating real from fake images.
        labels = tf.concat([tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0)
        print(f"Discriminator labels shape: {labels.shape}")

        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            print(f"Discriminator predictions shape: {predictions.shape}")
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

        # Sample random points in the latent space.
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim), seed=self.seed)
        random_vector_labels = tf.concat([random_latent_vectors, one_hot_labels], axis=1)

        # Assemble labels that say "all real images".
        misleading_labels = tf.zeros((batch_size, 1))
        print(f"Generator misleading labels shape: {misleading_labels.shape}")

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels)
            fake_image_and_labels = tf.concat([fake_images, image_one_hot_labels], axis=-1)
            predictions = self.discriminator(fake_image_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }

In [ ]:
# Instantiate and compile the ConditionalGAN model as before
cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

In [ ]:
cond_gan.fit(dataset, epochs=20)

Epoch 1/20
Real images shape: (None, 128, 96, 3)
One-hot labels shape: (None, 2)
Image one-hot labels shape: (None, 128, 96, 2)
Random latent vectors shape: (None, 128)
Random vector labels shape: (None, 130)
Generated images shape: (None, 128, 96, 3)
Combined images shape: (None, 128, 96, 5)
Discriminator labels shape: (None, 1)
Discriminator predictions shape: (None, 1)
Generator misleading labels shape: (None, 1)
